In [13]:
from pymongo import MongoClient
import pandas as pd
import re
import folium

In [14]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [15]:
db, coll = connectCollection('companies','companies')
print(coll.json)


Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'companies'), 'companies.json')


In [16]:
companies = list(coll.find({"$and":
                            [{"deadpooled_year":None},
                            {"total_money_raised":{"$exists": True,"$ne":"$0"}},
                            {"offices.latitude": {"$exists": True, "$ne": None}},
                            {"offices.longitude": {"$exists": True, "$ne": None}}]},
                          {"name":1, "founded_year":1, "offices":1, "category_code":1,
                           "total_money_raised":1}))
companies

[{'_id': ObjectId('52cdef7c4bab8bd675297d96'),
  'name': 'Gizmoz',
  'category_code': 'web',
  'founded_year': 2003,
  'total_money_raised': '$18.1M',
  'offices': [{'description': None,
    'address1': None,
    'address2': None,
    'zip_code': None,
    'city': 'Menlo Park',
    'state_code': 'CA',
    'country_code': 'USA',
    'latitude': 37.48413,
    'longitude': -122.169472}]},
 {'_id': ObjectId('52cdef7c4bab8bd675297d91'),
  'name': 'Geni',
  'category_code': 'web',
  'founded_year': 2006,
  'total_money_raised': '$16.5M',
  'offices': [{'description': 'Headquarters',
    'address1': '9229 W. Sunset Blvd.',
    'address2': '',
    'zip_code': '90069',
    'city': 'West Hollywood',
    'state_code': 'CA',
    'country_code': 'USA',
    'latitude': 34.090368,
    'longitude': -118.393064}]},
 {'_id': ObjectId('52cdef7c4bab8bd675297d8d'),
  'name': 'Digg',
  'category_code': 'news',
  'founded_year': 2004,
  'total_money_raised': '$45M',
  'offices': [{'description': None,
    'a

In [17]:
print(len(companies))

2690


In [18]:
companies1 = pd.DataFrame(companies)
companies1.head()

,_id,name,category_code,founded_year,total_money_raised,offices
0,52cdef7c4bab8bd675297d96,Gizmoz,web,2003.0,$18.1M,"[{'description': None, 'address1': None, 'addr..."
1,52cdef7c4bab8bd675297d91,Geni,web,2006.0,$16.5M,"[{'description': 'Headquarters', 'address1': '..."
2,52cdef7c4bab8bd675297d8d,Digg,news,2004.0,$45M,"[{'description': None, 'address1': '135 Missis..."
3,52cdef7c4bab8bd675297d99,Lala,games_video,NaN,$44.2M,"[{'description': 'Lala Headquarters', 'address..."
4,52cdef7c4bab8bd675297d98,Slacker,music,2006.0,$73.1M,"[{'description': '', 'address1': '16935 W. Ber..."


In [19]:
def getLocation(companies):
    location =[]
    for i in range(len(companies['offices'])):
        longitude = companies['offices'][i]['longitude']
        latitude = companies['offices'][i]['latitude']
        loc = {
            'type':'Point',
            'coordinates':[longitude, latitude]
        }
        location.append(loc)
    return location

In [25]:
'''for cp in companies:
    value = {"$set": {'location':getLocation(cp)}}
    coll.update_one(cp, value)
    print(cp)
    '''

{'_id': ObjectId('52cdef7c4bab8bd675297d96'), 'name': 'Gizmoz', 'category_code': 'web', 'founded_year': 2003, 'total_money_raised': '$18.1M', 'offices': [{'description': None, 'address1': None, 'address2': None, 'zip_code': None, 'city': 'Menlo Park', 'state_code': 'CA', 'country_code': 'USA', 'latitude': 37.48413, 'longitude': -122.169472}]}
{'_id': ObjectId('52cdef7c4bab8bd675297d91'), 'name': 'Geni', 'category_code': 'web', 'founded_year': 2006, 'total_money_raised': '$16.5M', 'offices': [{'description': 'Headquarters', 'address1': '9229 W. Sunset Blvd.', 'address2': '', 'zip_code': '90069', 'city': 'West Hollywood', 'state_code': 'CA', 'country_code': 'USA', 'latitude': 34.090368, 'longitude': -118.393064}]}
{'_id': ObjectId('52cdef7c4bab8bd675297d8d'), 'name': 'Digg', 'category_code': 'news', 'founded_year': 2004, 'total_money_raised': '$45M', 'offices': [{'description': None, 'address1': '135 Mississippi St', 'address2': '', 'zip_code': '94107', 'city': 'San Francisco', 'state_co

{'_id': ObjectId('52cdef7c4bab8bd6752980e4'), 'name': 'GeoSentric', 'category_code': 'mobile', 'founded_year': 2007, 'total_money_raised': '$24.1M', 'offices': [{'description': 'Amsterdam', 'address1': 'Keizersgracht 212', 'address2': '', 'zip_code': '1016 DX', 'city': 'Amsterdam', 'state_code': None, 'country_code': 'NLD', 'latitude': 37.09024, 'longitude': -95.712891}]}
{'_id': ObjectId('52cdef7c4bab8bd6752980e8'), 'name': 'RescueTime', 'category_code': 'web', 'founded_year': 2007, 'total_money_raised': '$920k', 'offices': [{'description': 'Pioneer Square', 'address1': '811 1st Ave', 'address2': 'Suite 480', 'zip_code': '98104', 'city': 'Seattle', 'state_code': 'WA', 'country_code': 'USA', 'latitude': 47.603577, 'longitude': -122.335333}]}
{'_id': ObjectId('52cdef7c4bab8bd6752980f3'), 'name': 'Dabble', 'category_code': 'games_video', 'founded_year': None, 'total_money_raised': '$350k', 'offices': [{'description': None, 'address1': '2342 Shattuck Ave. #276', 'address2': None, 'zip_cod

{'_id': ObjectId('52cdef7c4bab8bd675298672'), 'name': 'iovation', 'category_code': 'security', 'founded_year': 2004, 'total_money_raised': '$15M', 'offices': [{'description': None, 'address1': '111 SW 5th Avenue', 'address2': 'Suite 3200', 'zip_code': '97204', 'city': 'Portland', 'state_code': 'OR', 'country_code': 'USA', 'latitude': 45.52254, 'longitude': -122.675536}]}
{'_id': ObjectId('52cdef7c4bab8bd67529866e'), 'name': 'Zemanta', 'category_code': 'web', 'founded_year': 2007, 'total_money_raised': '$7.35M', 'offices': [{'description': 'London Office', 'address1': '107 Cheapside', 'address2': '', 'zip_code': 'EC2V 6DY', 'city': 'London', 'state_code': None, 'country_code': 'GBR', 'latitude': 51.5141566, 'longitude': -0.0934254}, {'description': 'New York Office', 'address1': '33 West 17th St', 'address2': 'Suite 903', 'zip_code': '10011', 'city': 'New York', 'state_code': 'NY', 'country_code': 'USA', 'latitude': 40.7387945, 'longitude': -73.9939213}, {'description': 'Ljubljana Offic

{'_id': ObjectId('52cdef7d4bab8bd675298f42'), 'name': 'OkCupid', 'category_code': 'web', 'founded_year': 2003, 'total_money_raised': '$6M', 'offices': [{'description': '', 'address1': '39 W 14th St', 'address2': 'Ste 502', 'zip_code': '10011-7403', 'city': 'New York', 'state_code': 'NY', 'country_code': 'USA', 'latitude': 40.7368158, 'longitude': -73.9955426}]}
{'_id': ObjectId('52cdef7d4bab8bd675298f77'), 'name': 'ComptTIA', 'category_code': 'nonprofit', 'founded_year': None, 'total_money_raised': '$70k', 'offices': [{'description': 'Corporate Headquarters', 'address1': '1815 S. Meyers Road', 'address2': 'Suite 300', 'zip_code': '60181-5228', 'city': 'Oakbrook Terrace', 'state_code': 'IL', 'country_code': 'USA', 'latitude': 41.849577, 'longitude': -87.99212}]}
{'_id': ObjectId('52cdef7d4bab8bd675298f53'), 'name': 'MarkLogic', 'category_code': 'enterprise', 'founded_year': 2003, 'total_money_raised': '$71.2M', 'offices': [{'description': 'HQ', 'address1': '999 Skyway Road', 'address2':

{'_id': ObjectId('52cdef7d4bab8bd675299cb9'), 'name': 'PeopleJar', 'category_code': 'web', 'founded_year': None, 'total_money_raised': '$1.61M', 'offices': [{'description': 'Los Angeles', 'address1': '', 'address2': '', 'zip_code': '90210', 'city': 'Los Angeles', 'state_code': 'CA', 'country_code': 'USA', 'latitude': 34.1030032, 'longitude': -118.4104684}]}
{'_id': ObjectId('52cdef7d4bab8bd675299cbf'), 'name': 'Brightstorm', 'category_code': 'games_video', 'founded_year': 2008, 'total_money_raised': '$6M', 'offices': [{'description': 'Headquarters', 'address1': '685 Market St. Street, Suite 270', 'address2': '', 'zip_code': '94105', 'city': 'San Francisco', 'state_code': 'CA', 'country_code': 'USA', 'latitude': 37.804619, 'longitude': -122.371203}]}
{'_id': ObjectId('52cdef7d4bab8bd675299cdb'), 'name': 'MarkTend', 'category_code': 'enterprise', 'founded_year': 2008, 'total_money_raised': '$150k', 'offices': [{'description': 'Home Base', 'address1': '4125 23rd St.', 'address2': '', 'zip

{'_id': ObjectId('52cdef7e4bab8bd67529ab4e'), 'name': 'Optichron', 'category_code': 'semiconductor', 'founded_year': 2003, 'total_money_raised': '$29M', 'offices': [{'description': 'HQ', 'address1': '4221 Technology Drive', 'address2': '', 'zip_code': '94538', 'city': 'Fremont', 'state_code': 'CA', 'country_code': 'USA', 'latitude': 37.5078425, 'longitude': -121.9554213}]}
{'_id': ObjectId('52cdef7e4bab8bd67529ab50'), 'name': 'Micron Technology', 'category_code': 'semiconductor', 'founded_year': 1978, 'total_money_raised': '$50.4M', 'offices': [{'description': '', 'address1': '8000 S. Federal Way', 'address2': 'P.O. Box 6', 'zip_code': '83707', 'city': 'Boise', 'state_code': 'ID', 'country_code': 'USA', 'latitude': 43.5926807, 'longitude': -116.2048973}]}
{'_id': ObjectId('52cdef7e4bab8bd67529ab51'), 'name': 'Siimpel Corporation', 'category_code': 'hardware', 'founded_year': 2000, 'total_money_raised': '$53.8M', 'offices': [{'description': '', 'address1': '400 E. Live Oak Avenue', 'add

{'_id': ObjectId('52cdef7e4bab8bd67529b7a7'), 'name': 'Arcxis Biotechnologies', 'category_code': 'biotech', 'founded_year': None, 'total_money_raised': '$3.44M', 'offices': [{'description': '', 'address1': '6920 Koll Center Parkway, Suite 215', 'address2': '', 'zip_code': '94566', 'city': 'Pleasanton', 'state_code': 'CA', 'country_code': 'USA', 'latitude': 37.6589038, 'longitude': -121.9011968}]}
{'_id': ObjectId('52cdef7e4bab8bd67529b7c4'), 'name': 'VasoGenix', 'category_code': 'biotech', 'founded_year': None, 'total_money_raised': '$400k', 'offices': [{'description': 'VasoGenix Pharma', 'address1': '8527 Bluejacket Street', 'address2': '', 'zip_code': '66214', 'city': 'Lenexa', 'state_code': 'KS', 'country_code': 'USA', 'latitude': 38.974099, 'longitude': -94.710676}]}
{'_id': ObjectId('52cdef7e4bab8bd67529b7d0'), 'name': 'Schoolnet', 'category_code': 'web', 'founded_year': 1998, 'total_money_raised': '$44.5M', 'offices': [{'description': 'Campusware', 'address1': '15303 Huebner Rd S

{'_id': ObjectId('52cdef7f4bab8bd67529c067'), 'name': 'Miles Electric Vehicles', 'category_code': 'automotive', 'founded_year': 2004, 'total_money_raised': '$56M', 'offices': [{'description': '', 'address1': '1601 Wilshire Blvd', 'address2': '', 'zip_code': '90403', 'city': 'Santa Monica', 'state_code': 'CA', 'country_code': 'USA', 'latitude': 34.028701, 'longitude': -118.486966}]}
{'_id': ObjectId('52cdef7f4bab8bd67529c065'), 'name': 'Solexant', 'category_code': 'cleantech', 'founded_year': 2006, 'total_money_raised': '$87M', 'offices': [{'description': '', 'address1': '2385 Bering Dr.', 'address2': '', 'zip_code': '95131', 'city': 'San Jose', 'state_code': 'CA', 'country_code': 'USA', 'latitude': 37.3825306, 'longitude': -121.9198306}]}
{'_id': ObjectId('52cdef7f4bab8bd67529c06a'), 'name': 'EnerG2', 'category_code': 'cleantech', 'founded_year': 2003, 'total_money_raised': '$12M', 'offices': [{'description': '', 'address1': '810 3rd Avenue, Suite 120', 'address2': '', 'zip_code': '981